In [1]:
%matplotlib inline
%gui qt
import linkerapp as linkerAPP
import linkerappUtil as Util
import linkerappGUI as GUI

In [ ]:
# select a working directory
wd = GUI.folderDialog()

In [ ]:
# select a cadnano deisgn file
json = GUI.fileDialog(wd=wd, filter="caDNAno (*.json)")

In [ ]:
psf = GUI.fileDialog(wd=wd, filter="topology (*.psf)")

In [ ]:
coor = GUI.fileDialog(wd=wd, filter="coordinates (*.dcd *.pdb)")

In [ ]:
mrc = GUI.fileDialog(wd=wd, filter="cryo-map (*.mrc)")

In [ ]:
seq = GUI.fileDialog(wd=wd, filter="sequence file (*.seq *.txt)")

# 2. parse filepaths and create Viewer instance
The viewer class contains both the data instances (design, map, model) and the selection functionality.
On creating the linkage between these fit and design is computed. this step might require a couple of seconds, depending on the design size.

In [ ]:
files = linkerAPP.Files(json, psf, coor, mrc, seq)
viewer = linkerAPP.Viewer(wd, files=files)

# 3. select subset of the atomic model
the viewer class supports different selection options:
* design-specific selection: subset specified by helix-id and base-position as in caDNAno designfile.
    * using selection widget -> option A
    * manual selection -> option B
* fit-specific selection: subset directly by MDAnalysis atomselection functionalities
    * example strand-specific selection: -> option C
    
    
In the end, all selections are translated into a selection of atom, stored as an MDAnalysis AtomGroup.   


### Option A: selection with widget 
selection: selected helices (clicked) are displayed in dark grey. the "base" slider selects the base-position range.
context: slider for the area surrounding a selected atom for mrc-cropping (in Å)

<br />

---

the content of the widget is evaluated by executing

``helixandbase, context = viewer.eval_sliders(*sliders)`` 
which returns a tuple containing two lists([helix-ids],[base_ids]) and the context value

the list-tuple can be translated into an AtomGroup by executing:
``atoms_selection, color_dict = viewer.select_by_helixandbase(*helixandbase)``

the method returns the AtomGroup ``atoms_selection`` 

---

(it also returns the dictionary ``color_dict`` containing information about staple coloring in the .json file, that can be used to color staples in the ngl_view widget)

In [ ]:
sliders = viewer.select_widget()

In [ ]:
helixandbase, context = viewer.eval_sliders(*sliders)
atoms_selection, color_dict = viewer.select_by_helixandbase(*helixandbase)

In [ ]:
print(helixandbase)
len(atoms_selection.residues)

### Option B: manual selection
``viewer.select_by_helixandbase(helices, base)`` can also be called directly using two lists, specifying the helix-ids and base positions.

the method can also be applied to multiple subsets of the data, as MDAnalysis AtomGroups can be combined to realize more complex selections. (more details see option C)

In [ ]:
## example Option B:
atoms_selection, color_dict = viewer.select_by_helixandbase([12,13], range(40,68))

### option C: Fit secific selection
generate atomselection using mdAnalysis. 
the MDAnalysis baseclass is available via ``viewer.linker.fit.u``

documentation:(https://www.mdanalysis.org/docs/documentation_pages/selections.html)
NOTES:
* AtomGroups are combinable using set-syntax (f.e. selection = selectionA + selectionB)
* AtomGroups can be inverted by subtraction them from the full system accessible via .universe.atoms attribute. this is usefull for creating masks for multibody and local scanning refinement processes.
* (design-specific staple coloring is not compatible with this option)

In [ ]:
## example Option C: all double-stranded segments with staples beginning with the sequence ATCG

"""select all staples beginning with a specific sequence,
via the intersection of all staples with the correct base condition
start with all segments and filter with additional conditions
""" 
u = viewer.linker.fit.u
ATCG = u.segments
for idx, X in enumerate(["ADE", "THY", "CYT", "GUA"]):
    ATCG = ATCG.segments & u.select_atoms("resname {} and resid {}".format(X, idx+1)).segments
atoms_staple  = ATCG.segments.atoms

"""select all paired scaffold bases using the Base-Pair list generated from the design in viewer.linker.link
start with empty group and iteratively add all scaffold bases
"""
atoms_scaffold = Util.AtomGroup([],u)
for residue in atoms_staple.residues:
    watson_crick_base = viewer.linker.link.Fbp_full.get(residue.resindex, None)
    if watson_crick_base:
        atoms_scaffold += u.residues[watson_crick_base].atoms

# combine scaffold and staple atoms to  form the final atom_selection
atoms_selection = atoms_scaffold + atoms_staple
context = 3

# 3. Cropping and Zoning 
the generated AtomGroup can be used for
* generating a pdb of te subset represented by the AtomGroup
* creating a new mrc-map that only contains data in the vicinity (context) of the selected atoms 

<br />

---
create subset MRC-file

``viewer.writemrc(atoms_selection, wd, name, [context=context, cut_box=True])``

cut_box: if `True`, remove zero-padding from volume

---

create subset pdb-file (chimeraX compatible)

``viewer.writepdb(atoms_selection, wd, name, [single_frame=True, frame=-1, chimeraX=True])``

single_frame: if `False`, create multiframe pdb for full trajectory (multiframe can take a couple of minutes)

frame: frame index to be saved, default -1 = last (only for .dcd file as coordinate-file)\

chimeraX: if `True`, also create chimeraX compatible pdb-file ("name_chim.pdb")

In [ ]:
#choose output name
name = "slice-test2"

In [ ]:
#create subset MRC-file
viewer.writemrc(atoms_selection, name, context=context, cut_box=False)

In [ ]:
#create subset pdb-file (chimeraX compatible)
viewer.writepdb(atoms_selection, name, singleframe=True, frame=-1, chimeraX=True)

# 4.  display selection
NOTE: requires saved mrc file of the selection
* isovalue: isosurface threshold
* backbone: `True` -> only backbone atoms displayed (faster)
* color_dict: `None` -> no staple coloring (faster)

for detailed analysis Applications like UCSF-ChimeraX or VMD are recomended

In [ ]:
mrc = viewer.project.folder / "{}.mrc".format(name)
Util.fit_widget(atoms_selection, mrc, isolevel=1.35, backbone=True, color_dict=None)